In [3]:
import datetime
import json
import requests
import sys
import time
import geopandas
from path import Path

In [4]:
# path configuration
working_directory = Path.getcwd()

inputs_path = working_directory / 'inputs'
outputs_path = working_directory / 'outputs'

### Initial Setup

In [5]:
# Set StreetLight key and email
StlKey = "DgO7zCdYednFilSOR1dTX7jsmsdcoz31"
email = "chris.ryan@hdrinc.com"

In [7]:
now = datetime.datetime.utcnow().isoformat()

In [8]:
ZONE_SET_NAME = "Two_Neighborhoods_{}".format(now)
ANALYSIS_NAME = "OD_for_TN_{}".format(now)


def print_response(response):
    print("response code: {}".format(response.status_code))
    print("response body: {}".format(response.content))

In [10]:
# ----------------------------------------------------------------------------------------
# Create a Zone Set.
# ----------------------------------------------------------------------------------------

ZONE_SET_REQUEST = {
    "insight_login_email": email,
    "zone_set_name": ZONE_SET_NAME,
    "zones": {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "id": 1,
                    "name": "Mission",
                    "is_pass": 0,
                    "direction": None,
                },
                "geometry": {
                    "type": "MultiPolygon",
                    "coordinates": [
                        [
                            [
                                [-122.426698258661972, 37.769562689936315],
                                [-122.423394859041892, 37.772083876030344],
                                [-122.42225575572462, 37.770418101996292],
                                [-122.411206453547067, 37.769427623969634],
                                [-122.406991771273155, 37.769067446852375],
                                [-122.404656609472752, 37.767716767038294],
                                [-122.405169205965521, 37.762628984940662],
                                [-122.406593085112107, 37.760557752167074],
                                [-122.405795712790038, 37.758441432691242],
                                [-122.403232730326167, 37.75722564731312],
                                [-122.402549268335804, 37.751821915004783],
                                [-122.403346640657901, 37.749390106706535],
                                [-122.407561322931784, 37.748399347080543],
                                [-122.424875693354338, 37.74781389197571],
                                [-122.426698258661972, 37.769562689936315],
                            ]
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "id": 2,
                    "name": "Financial District",
                    "is_pass": 1,
                    "direction": None,
                },
                "geometry": {
                    "type": "MultiPolygon",
                    "coordinates": [
                        [
                            [
                                [-122.405425504211919, 37.798033588378779],
                                [-122.398476973976571, 37.798933675051543],
                                [-122.396654408668923, 37.799698740100226],
                                [-122.397024617247041, 37.79929370664977],
                                [-122.396768319000643, 37.798911173018389],
                                [-122.395828558763895, 37.797898574431919],
                                [-122.393607307295227, 37.799113691070012],
                                [-122.392610591892605, 37.797876072083447],
                                [-122.394233814119715, 37.79690846455324],
                                [-122.393037755636598, 37.795670808599994],
                                [-122.3913575782436, 37.796278387822021],
                                [-122.390987369665496, 37.795693311702443],
                                [-122.392439726395011, 37.794500642185817],
                                [-122.389278714689581, 37.791462623416393],
                                [-122.401182344355107, 37.781965196242638],
                                [-122.405824190372982, 37.785701296639232],
                                [-122.406222876534017, 37.785723802695827],
                                [-122.407134159187834, 37.790337399578668],
                                [-122.404058580231194, 37.790764986655553],
                                [-122.405425504211919, 37.798033588378779],
                            ]
                        ]
                    ],
                },
            },
        ],
    },
}


In [11]:
CREATE_ANALYSIS_REQUEST = {
    "insight_login_email": email,
    "analysis_name": ANALYSIS_NAME,
    "analysis_type": "OD_Analysis",
    "travel_mode_type": "All_Vehicles_LBS_Plus",
    "description": "",
    "oz_sets": [{"name": ZONE_SET_NAME}],
    "dz_sets": [{"name": ZONE_SET_NAME}],
    "day_types": "All Days|17, Weekday|14, Weekend Day|67",
    "day_parts": "All Day|0023, Early AM|0005, Peak AM|0609, Mid-Day|1014, Peak PM|1518, Late PM|1923",
    "trip_attributes": False,
    "traveler_attributes": False,
}

In [13]:
resp = requests.post(
    "https://insight.streetlightdata.com/api/v2/analyses",
    headers={"content-type": "application/json", "x-stl-key": StlKey},
    data=json.dumps(CREATE_ANALYSIS_REQUEST),
)

print_response(resp)

if resp.status_code == 201:
    print("Created Analysis successfully.")
else:
    print("Error creating Analysis.")
    sys.exit(1)

SSLError: HTTPSConnectionPool(host='insight.streetlightdata.com', port=443): Max retries exceeded with url: /api/v2/analyses (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)')))

In [ ]:
# Set tmap_mode (if needed)
# tmap_mode("view")

proj = 6595

bufferSize = 5280 * 3
gridSize = 5280 * 0.33

# Read in shapefiles for stationPoints and stationArea
stationPoints = gpd.read_file("shape/stationPoints.shp")
stationPoints = stationPoints.to_crs(epsg=proj)
stationPoints = stationPoints.sort_values(by="name")

stationArea = gpd.read_file("shape/RailStationAreas.shp")
stationArea = stationArea.to_crs(epsg=proj)
stationArea = stationArea[["PropName"]].rename(columns={"PropName": "name"})
stationArea = gpd.sjoin(stationPoints, stationArea, op="within")[["name", "type"]]
stationArea = stationArea.sort_values(by="name")

# Map check
basemaps = ["CartoDB.Positron", "OpenStreetMap.Mapnik", "Esri.WorldImagery"]
stationArea.plot(color="orange", alpha=0.5)
stationPoints.plot(column="type", legend=True, cmap="Set1")

# NOTE: Confirm that both GeoDataFrames have the same number of locations and are in the same order for 'name'
(stationPoints["name"] == stationArea["name"]).all()


### Define each zone

In [ ]:
# Create StL Zone Sets ----------------------------------------------------

# Create Station Area Zones
stationArea_ZS = stationArea[["name"]].copy()
stationArea_ZS["id"] = range(1, len(stationArea_ZS) + 1)
stationArea_ZS["is_pass"] = 0
stationArea_ZS["direction"] = None
stationArea_ZS = stationArea_ZS.to_crs(epsg=4326)
stationArea_ZS = stationArea_ZS.set_z(0)


In [ ]:
# Convert to JSON
zone_set_name = "StationAreas2021-09-17"
zones = json.loads(stationArea_ZS.to_json())

body = {
    "insight_login_email": email,
    "geom_type": "polygon",
    "zone_set_name": zone_set_name,
    "zones": zones
}

# Convert to final JSON file
bodyJSON = json.dumps(body)

In [ ]:
# API call to create Zone Set
def api_call(url, data):
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response

url = f"https://insight.streetlightdata.com/api/v2/zone_sets?key={StlKey}"
api_call(url, bodyJSON)

In [ ]:
# Loop to create hex grid zone set for each station area
allGrid = pd.concat([
    buffer[buffer.sindex.query(buffer, predicate="intersects")]
    .difference(stationArea.loc[_]["geometry"])
    .rename(columns={"geometry": "name"})
    .reset_index(drop=True)
    .assign(id=range(1, len(grid) + 1), is_pass=0, direction=None)
    .set_z(0)
    for _, station in stationPoints.iterrows()
], ignore_index=True)

# Convert to JSON and make API call to create Zone Set
for _, station in stationPoints.iterrows():
    buffer = station.geometry.buffer(bufferSize)
    grid = buffer[buffer.sindex.query(buffer, predicate="intersects")]
    grid = grid.difference(stationArea.loc[_]["geometry"])
    grid = grid.rename(columns={"geometry": "name"}).reset_index(drop=True)
    grid["id"] = range(1, len(grid) + 1)
    grid["is_pass"] = 0
    grid["direction"] = None
    grid = grid.set_z(0)

    allGrid = allGrid.append(grid)

    zone_set_name = station["name"] + str(grid["id"].iloc[0])
    zones = grid.to_dict(orient='records')

    body = {
        "insight_login_email": email,
        "geom_type": "polygon",
        "zone_set_name": zone_set_name,
        "zones": zones
    }

    url = f"https://insight.streetlightdata.com/api/v2/zone_sets?key={StlKey}"
    response = requests.post(url, json=body)

allGrid.plot(alpha=0.5)

# Save data for Step 2.0
allGrid.to_file("shape/stationArea.geojson", driver="GeoJSON")